# BabyZK (crypto)

In this challenge, we have a safe prime $p$, a degree-15 polynomial $m$ in $\mathbb{F}_p$, and a generator $g$ of $\mathbb{F}_p^\times$, none of which are revealed to us. Instead, we have an oracle that can tell us the value of $g^{m(x)}$ for any $x$ we give it, but only 17 times. And then, in turn, we have to prove that we now know the value of $g^{m(x)}$ for all $x$.

The first self-evident fact we use is that we know 15 pairs of ($x$, $m(x)$), and we can obtain the values of all $m(x)$ as a linear combination of these 15. The linearity here is important, since we can "lift" the linearity into something we can calculate on our end, i.e.
$$g^{a m(x) + b} = \left(g^{m(x)}\right)^a g^b.$$

Except we cannot actually calculate the RHS of that equation without knowing $p$, so we will need to leak $p$ somehow. For a start, if we ignore the "mod-p"-ness of it we can still calculate the RHS but end up with something very large. And then we can compare this to an actual 16th value of $g^{m(x)}$ queried to the oracle, then we have two numbers that are known to be equal mod $p$. The caveat here is we cannot actually take a negative power, but we can always just move that to the other side.

So, a 16th query gives us a multiple of $p$, and if we are lucky we can factorise out small factors. Otherwise, we are given a 17th query anyway so we can obtain yet another multiple of $p$ this way. Now we can take the gcd of these two, and are much more likely to get the true $p$ (maybe still factorise out small factors just in case).

In [1]:
from pwn import *
from sage.all import *
from tqdm import trange

In [2]:
mat = matrix([[a**i for i in range(15)[::-1]] for a in range(17)])
ker = matrix(mat.kernel().basis())
ker

[     1      0   -120   1120  -5460  17472 -40040  68640 -90090  91520 -72072  43680 -20020   6720  -1560    224    -15]
[     0      1    -15    105   -455   1365  -3003   5005  -6435   6435  -5005   3003  -1365    455   -105     15     -1]

We have a 2-dimensional kernel as expected, and for each element in the basis we can take the positives to one side and negatives to the other, multiply them out, and get a difference that is guaranteed to be a multiple of $p$.

That said, we are potentially dealing with some large numbers, e.g. there's a 90090 in there, and a 1024-bit number taken to the power of 90090 is huge. We use LLL to try to reduce the basis to something more tangible.

In [3]:
ker = ker.LLL()
ker

[    1   -14    90  -350   910 -1638  2002 -1430     0  1430 -2002  1638  -910   350   -90    14    -1]
[    0     1   -15   105  -455  1365 -3003  5005 -6435  6435 -5005  3003 -1365   455  -105    15    -1]

There, much better. All that's left is to get our 17 pairs (we can just use 0 to 16), and be on our way!

In [4]:
sh = remote('babyzk-01.hfsc.tf',4590)
sh.recvuntil(b'Exit.\n')
def getval(n):
    sh.sendlines([b'1', str(n).encode()])
    return int(sh.recvline().decode()[2:-1:])

proofs = [getval(i) for i in range(17)]
print('got proofs')

tmp0 = product(a**i for i,a in zip(ker[0], proofs) if i > 0) - product(a**-i for i,a in zip(ker[0], proofs) if i < 0)
tmp1 = product(a**i for i,a in zip(ker[1], proofs) if i > 0) - product(a**-i for i,a in zip(ker[1], proofs) if i < 0)
p = gcd([tmp0, tmp1])
for i in range(2, 10000):
    while p % i == 0:
        p = p // i
print(f'{p.nbits()=}')
print(f'{p=}')
F = GF(p)

[x] Opening connection to babyzk-01.hfsc.tf on port 4590
[x] Opening connection to babyzk-01.hfsc.tf on port 4590: Trying 159.65.24.100
[+] Opening connection to babyzk-01.hfsc.tf on port 4590: Done
got proofs
p.nbits()=1025
p=186545829008915486081349235204194662613552063467070259300995581148452197614154813603872996153043490077697203112683470125198616204854218030130987551848701864290627846430235305557522066407877425140407944127150384706950180268511143513804663027126983176152898976672461383039178578788291326387946167353596286766407


In [5]:
sh.sendline(b'2')
for _ in trange(100):
    want = int(sh.recvline())
    vec = vector([want**i for i in range(15)[::-1]])
    soln = mat.solve_left(vec)
    F = GF(p)
    finalanswer = product(F(a)**i for i,a in zip(soln, proofs))
    sh.sendline(str(finalanswer).encode())

sh.sendline(b'3')    
sh.recvall()

100%|██████████| 100/100 [07:14<00:00,  4.34s/it]

[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 40B
[+] Receiving all data: Done (40B)
[*] Closed connection to babyzk-01.hfsc.tf port 4590


b'midnight{n0t_h4rd_3ven_w1th0ut_modulu5}\n'